# Reading joblib and sensitivity analysis function

In [12]:
import base64
import datetime
import io

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import numpy as np

import pandas as pd
import plotly.graph_objects as go


In [13]:
path = "C:/Users/sdam/Documents/IESEG/Hackathon/wetransfer-f36ccf/Application/data_app/"

In [14]:
#importing csv file where some data is stored 
df = pd.read_csv(path + 'final_test_15var.csv',index_col= False)
#information about country
country = pd.read_csv(path + 'country.csv',index_col= False)
country['Literal'].loc[[41]] = 'NA'
#information about occupacion
occupacion = pd.read_csv(path + 'occupacion.csv',index_col= False,encoding='latin-1')
#information about province
province = pd.read_csv(path + 'provincia.csv',index_col= False,encoding='latin-1')
#information about province
email = pd.read_csv(path + 'email.csv',index_col= False)
#information about code bank
code_bank = pd.read_csv(path + 'code_bank.csv', index_col=False)
#informaiton about the rfm_segment
#rfm_segment = pd.read_csv("./rfm_segment.csv")
#dataset with exstra observations
test6 = pd.read_csv(path + "test6.csv")

In [15]:
#importing the model
from joblib import load
pipeline = load(path + 'lrBest_final15.joblib')

In [16]:
pipeline

LogisticRegression(C=29.7, class_weight={0: 1, 1: 18}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
#fucntion to create the sensitivity table
def sensitivity(x_test_proba,y_test, predi_proba, loan_amount_test ,marginal_profit = [0.03, 0.05, 0.07, 0.1, 0.15], marginal_profit2 = ['Three', 'Five', 'Seven', 'Ten', 'Fifteen'],  total = ['Total Profit (3%)', 'Total Profit (5%)', 'Total Profit (7%)', 'Total Profit (10%)', 'Total Profit (15%)']): # Populate the arrays for the strategy table with a for loop
    x_test_proba = x_test_proba.copy()
    x_test_proba['SIP3'] = y_test
    x_test_proba['preds'] = predi_proba
    x_test_proba['loan_amount2'] = loan_amount_test
    accept_rates = [1.0, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25,
     0.2, 0.15, 0.1, 0.05]
    thresholds = []
    bad_rates = []
    badPred_rates = []
    num_accep_loans = []
    total_loan_amount = []
    for i in accept_rates:
        # Calculate the threshold for the acceptance rate
        thresh = np.quantile(x_test_proba['preds'], i).round(5)
        # Add the threshold value to the list of thresholds
        thresholds.append(np.quantile(x_test_proba['preds'], i).round(5))
        # Reassign the loan_status value using the threshold
        x_test_proba['preds_num'] = x_test_proba['preds'].apply(lambda x: 1 if x > thresh else 0)
        # Create a set of accepted loans using this acceptance rate
        accepted_loans = x_test_proba[x_test_proba['preds_num'] == 0]
        # Calculate and append the bad rate using the acceptance rate
        bad_rates.append(np.sum((accepted_loans['SIP3']) / len(accepted_loans['SIP3'])).round(5))
                
        num_accep_loans.append(len(x_test_proba[x_test_proba['preds'] < np.quantile(x_test_proba['preds'], i)]))    
        
        # Amount of loan per quantile
              
        a = x_test_proba[x_test_proba['preds'] < np.quantile(x_test_proba['preds'], i)]
        total_loan_amount.append(a['loan_amount2'].sum())
                
        # Number of accepted loan per quantile
    strat_df = pd.DataFrame(zip(accept_rates, thresholds, bad_rates, num_accep_loans, total_loan_amount ),
                        columns = ['AcceptanceRate','Threshold','Bad Rate', 'Num Accept Loans', 'Portafolio'])
    # Potential Loss
    strat_df['potential_loss'] = 0
    strat_df['potential_loss'] = strat_df['Portafolio']*strat_df['Bad Rate']
    
    
    strat_df['Potential saving'] = 0 # Potential saving
    # Loan not granted
    strat_df['Loan not granted'] = 0 # Loan not granted
    
    for i in range(strat_df['potential_loss'].count()):
        strat_df['Potential saving'][i] = strat_df['potential_loss'][0]-strat_df.iloc[i,5]
        strat_df['Loan not granted'][i] = strat_df['Portafolio'][0]-strat_df.Portafolio[i]
   
    strat_df['Total not granted missclasified'] = strat_df['Loan not granted'] - strat_df['potential_loss'] 
        
   
    for i in range(len(marginal_profit)):
        strat_df[marginal_profit2[i]]  = strat_df['Potential saving']-strat_df['Total not granted missclasified']* marginal_profit[i] 
    strat_df.iloc[0:1,8:14] = 0   
    for i in range(len(marginal_profit)):
        strat_df[total[i]]  = (strat_df['Portafolio']-strat_df['potential_loss'])*marginal_profit[i]+strat_df[marginal_profit2[i]]
        strat_df.iloc[0:1,i+14] = (strat_df['Portafolio']-strat_df['potential_loss'])*marginal_profit[i]
        
    
    strat_df.iloc[:,3:] = round(strat_df.iloc[:,3:],0)
    # Print the entire table
    return(strat_df)

In [19]:
# Reading Data
X_test = pd.read_csv(path+
    "X_test_opendata.csv", sep=',')
X_test = X_test.iloc[:,1:]
y_test = X_test['SIP3']

# Saving loan amount for the sensitivy analysis. It has to be separated from the X_test
loan_amount_test = X_test['loan_amount']
X_test.drop(columns=['SIP3'], inplace= True)


# Replacing inifinties with 0 
X_test.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
Ind = X_test.filter(regex='Ind')

X_test.drop(columns=Ind, axis=0, inplace=True)

# **********  Scaling  ***************  
from sklearn.preprocessing import scale    #Standarize
names = X_test.columns
#X_train =  pd.DataFrame(scale(X_train), columns = names)
X_test =  pd.DataFrame(scale(X_test), columns = names)


#Running the imported model on the test set for sensitivity analysis
x_test_proba = pipeline.predict_proba(df)[:,1]

predi_proba = pipeline.predict(df)

#Running the sensitivity analysis function to generate the sensitivity table
sense_df =sensitivity(X_test,y_test, x_test_proba, loan_amount_test ) 
sense_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,AcceptanceRate,Threshold,Bad Rate,Num Accept Loans,Portafolio,potential_loss,Potential saving,Loan not granted,Total not granted missclasified,Three,Five,Seven,Ten,Fifteen,Total Profit (3%),Total Profit (5%),Total Profit (7%),Total Profit (10%),Total Profit (15%)
0,1.00,0.97507,0.03643,60549,150130150.0,5469241.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4339827.0,7233045.0,10126264.0,14466091.0,21699136.0
1,0.95,0.63580,0.03195,57522,141582300.0,4523554.0,945686,8547850,4024296.0,824957.0,744471.0,663985.0,543256.0,342042.0,4936720.0,7597408.0,10258098.0,14249131.0,20900853.0
2,0.90,0.57810,0.02942,54495,133645300.0,3931845.0,1537396,16484850,12553005.0,1160806.0,909746.0,658686.0,282095.0,-345555.0,5052210.0,7395419.0,9738628.0,13253441.0,19111464.0
3,0.85,0.53510,0.02697,51467,125929500.0,3396319.0,2072922,24200650,20804331.0,1448792.0,1032705.0,616619.0,-7511.0,-1047728.0,5124788.0,7159365.0,9193942.0,12245807.0,17332250.0
4,0.80,0.49871,0.02479,48440,118463400.0,2936708.0,2532533,31666750,28730042.0,1670632.0,1096031.0,521430.0,-340471.0,-1776973.0,5136432.0,6872366.0,8608298.0,11212198.0,15552030.0


# APP Code

In [23]:
import base64
import datetime
import time
import io
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from joblib import load

woe_map2 = load(path+'woe.joblib')
test6 = pd.read_csv(path+"test6.csv")

def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')),index_col=False)
            global gbdf2
            gbdf2 = df.copy()
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
            gbdf2 = df.copy()
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])
    global pred
    df['proba'] = pipeline.predict_proba(gbdf2)[:,1]

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            data=df.head(10).to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ]
    )







app = dash.Dash(__name__
               )

#creating the layout of the application
app.layout = html.Div([ 
    dcc.Tabs([
        
        
        dcc.Tab(
            label= 'Credit Strategy & Sensitivity Analysis',#title of the tab
            children = [
                html.H1(children='Sensitivity Analysis Table'),
                html.Div(id = 'sensitivity_text',
                         #create a markdawn to give some information about the sensitivity analysis
                         children='''
                         Below you find a table that represent the potential profit of using the model.
                         By selecting the correct profit margin of a loan. you get an overview of the total profit made for the corresponding threshold.
                         In this way, you can see the most optimal threshold for a certain profit margin.
                         '''),
                html.H4(children='Enter Profit Margin'),#sub title
                #creating the display of the sensitivity analysis
                html.Div(id = 'profit_margin_drop',
                         style= {'width': '50%'},
                         children = [
                             dcc.Dropdown(id = 'profit_margin',
                                #giving the possible profit margins
                                 options=[{'label': i, 'value': i} for i in sense_df.iloc[:,14:].columns] ,
                                #text in the box that says to select the correct profit margin          
                                 placeholder="Select a Profit Margin Percentage",
                                 value= 1,
                                #style of the box
                                 style={
                                     'width': '60%',
                                    'height': '60px',
                                    'lineHeight': '60px',
                                    'lineWidth': '60px',
                                    'display':'block',
                                    "margin-left": "auto",
                                     "margin-right": "auto",
                                    'borderWidth': '1px',
                                    'borderStyle': 'dashed',
                                    'borderRadius': '5px',
                                    'textAlign': 'center'
                                     },
                                 )
                             ]
                        ),
                #creating the display settings for displaying the sensetivity table
                html.Div(id = 'sense_div' ),
                html.Div(id='table_sense',
                         style= {'width': '50%'},
                         children=[
                             #title
                             html.H4(children='Total Profit'),
                             dash_table.DataTable(
                                 id='sense_table_dropdown',
                                 #data of the table
                                 data = sense_df.to_dict('records'),
                                 #all the columns of the table
                                 columns =[{"name": i, "id": i} for i in sense_df],
                                 #don't make it editable
                                 editable=False,
                                 #style of the table
                                 style_cell={
                                     'width': '35%',
                                     'textOverflow': 'ellipsis',
                                     'margin-right': 'auto',
                                     'margin-left': '10%',
                                     'overflow': 'hidden'
                                     },
                             )
                         ]
                        )
                ]
        ),
        #creating new tab for the individual customer prediction
        dcc.Tab(label = 'Individual Customer Prediction',
                children = [
                    #title
                    html.H1(children='Enter values for prediction of default'),
                    html.Div(id = 'Input_variables_div1',
                             style= {'width': '50%','margin-left':'none','margin-right':'auto',
                                    'display':'block'},#style of the tab
                             children = [
                                 html.H4(children='Enter Housing'),
                                 #creating the first box for housing
                                 dcc.Dropdown(id = 'vivienda',
                                              #possible options to select for housing, white a corresponding number as used in the model
                                              options=[
                                                  {'label': 'Alojado de forma gratuita', 'value': 1},
                                                  {'label': 'Hipoteca', 'value': 2},
                                                  {'label': 'Empresa', 'value': 3},
                                                  {'label': 'Alquilado', 'value': 4},
                                                  {'label': 'Familia', 'value': 5},
                                                  {'label': 'Propia', 'value': 6},
                                                  {'label': 'Hotel', 'value': 7},
                                                  {'label': 'Caravana', 'value': 8},
                                                  {'label': 'Otra vivienda', 'value': 9},
                                                  {'label': 'N.I.', 'value': 99}
                                                  ],
                                              #what you see in te box before selecting a value
                                              placeholder="Select Housing",
                                              #style of the box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #creating a box for the ban entity code
                                 html.H4(children='Enter Bank Entity Code '),
                                 #dcc.Input(
                                 #    id="COD_BQE_PQI_4_num", type="number",
                                 #    debounce=False, placeholder="Bank Entity Code",
                                 #),

                                 dcc.Dropdown(id = 'COD_BQE_PQI_4_num',
                                              #creating the different options for the email from a previous imported csv file
                                              options=[{'label': i, 'value': i} for i in code_bank.Descriptivo.unique()],
                                              #what in the box is written before selecting email domain
                                              placeholder="Enter Bank Entity Code",
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #creating a box to for entenry the channel of the insttrucion
                                 html.H4(children='Enter creation channel of the Instruction'),
                                 dcc.Dropdown(id = 'Origen',
                                              #possible options
                                              options=[
                                                  {'label': 'Web', 'value': 1},
                                                  {'label': 'Teléfono', 'value': 0},
                                              ],
                                              #what you see in the box before selecting a value
                                              placeholder="Select Channel",
                                              #the style of the channel box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #creating box for the number of participants at the loan 
                                 html.H4(children='Enter Number of main participants'),
                                 dcc.Dropdown(id = 'num_tit',
                                              #possible options
                                              options=[
                                                  {'label': 1, 'value': 1},
                                                  {'label': 2, 'value': 2},
                                              ],
                                              #what you see in the box before selecting a value
                                              placeholder="No. of main participants",
                                              #the style of the participant box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #making the box for age
                                 html.H4(children='Enter Age'),
                                 dcc.Input(
                                     id="edad", type="number",
                                     debounce=False, placeholder="Enter Age",
                                 ),
                                 #making the box for Nationality code
                                 html.H4(children='Enter Nationality Code'),
                                 dcc.Dropdown(id = 'pais_cat',
                                              #importing the options out of the previous imported csv file country
                                              options=[{'label': country['Descriptivo'].loc[[i]].item(), 'value': country['Literal'].loc[[i]].item()} for i in range(len(country))],
                                              
                                              placeholder="Select Country",
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #creating the box occupation
                                 html.H4(children='Enter Occupation'),
                                 dcc.Dropdown(id = 'occu_cat',
                                              #importing the options out of the previous imported csv file occupacion
                                              options=[{'label':occupacion['Descriptivo'].loc[[i]].item(), 'value': occupacion['Literal'].loc[[i]].item()} for i in range(len(occupacion)) ],
                                              
                                              placeholder="Select occupation",
                                              style={
                                                  'width': '85%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '60px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'left'
                                                  },
                                              ),
                                 ]
                        ),
                    #creating of the visualtisation for displaying the default percentage
                    html.Div(id = 'pred_circle',
                             #setting where do the graph need to display
                             style= {'width': '50%','display':'block', "margin-left": "auto",'right':'4%',
                                     
                                    'top':'15%','vertical-align': 'text-top','position':'fixed'},
                             children=[
                                 #title
                                 html.H4(children='Percentage of Default'),
                                 html.Div(id = 'pred_text',
                                          children='''
                                          .
                                          '''),
                                 #graph settings
                                 dcc.Graph(id = 'pred_pie',
                                           style={
                                               'width': '70%',
                                               'height': '30%',
                                               'lineHeight': '60px',
                                               'lineWidth': '60px',
                                               'display':'block',
                                               'margin-left': 'auto',
                                               'margin-right': 'auto',
                                               'borderWidth': '1px',
                                               'borderStyle': 'hidden',
                                               'borderRadius': '5px',
                                               'textAlign': 'center'
                                               },
                                           ),
                                 ]
                             ),
                    
                    html.Div(id = 'input_var_div_2',
                             style= {'width': '50%','margin-left':'none',
                                     'margin-right':'auto','margin-top':'none',
                                     'display':'block',
                                     'position':'absolute',
                                     },
                             children = [
                                 #making the province box
                                 html.H4(children='Select Province'),
                                 dcc.Dropdown(id = 'prov_cat',
                                              #importing the different provinces form a imported csv file province
                                              options=[{'label':province['Descriptivo'].loc[[i]].item(), 'value': province['Literal'].loc[[i]].item()} for i in range(len(province)) ],
                                              #text in the box before selecting a value
                                              placeholder="Select province",
                                              #style of the box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '60px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'left'
                                                  },
                                              ),
                                 #making the gender box
                                 html.H4(children='Select Gender'),
                                 dcc.Dropdown(id = 'gender',
                                              #creating the different options for gender
                                              options=[
                                                  {'label': 'Femenino', 'value': 'F'},
                                                  {'label': 'Masculino', 'value': 'M'},
                                              ],
                                              #text in the box before slecting the gender
                                              placeholder="Select Gender",
                                              #style of the gender box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #making the email box
                                 html.H4(children='Select Email Domain'),
                                 dcc.Dropdown(id = 'email_domain',
                                              #creating the different options for the email from a previous imported csv file
                                              options=[{'label': i, 'value': i} for i in email.Descriptivo.unique()],
                                              #what in the box is written before selecting email domain
                                              placeholder="Select email domain",
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #making the type of job box
                                 html.H4(children='Select Type of Job'),
                                 dcc.Dropdown(id = 'BLZ_OCCUPATION_1',
                                              #creating diferent options
                                              options=[
                                                  {'label': 'Asalariado', 'value': 'A'},
                                                  {'label': 'Funcionario', 'value': 'B'},
                                                  {'label': 'Parado', 'value': 'C'},
                                                  {'label': 'Estudiante', 'value': 'E'},
                                                  {'label': 'Jubilado/Pensionista', 'value': 'O/R'},
                                                  {'label': 'Ama de casa', 'value': 'P'},
                                                  {'label': 'Pension Contributiva', 'value': 'T'},
                                                  
                                              ],
                                              #what in the box is written before slecting a job
                                              placeholder="Select type of job",
                                              #style of the job box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #making the type of contract box
                                 html.H4(children='Select Type of Job Contract'),
                                 dcc.Dropdown(id = 'ord_tipcontr1',
                                              #creating the options for the type box
                                              options=[
                                                  {'label': 'Indefinido', 'value': 1},
                                                  {'label': 'Discontinuo', 'value': 2},
                                                  {'label': 'Temporal', 'value': 3},
                                                  {'label': 'Eventual/Interinidad', 'value': 4},
                                                  {'label': 'Mercantil', 'value': 5},
                                                  {'label': 'Aprendiz/Prácticas', 'value': 6},
                                                  {'label': 'Formación', 'value': 7},
                                                  {'label': 'N.I.', 'value': 99},
                                                  {'label': 'Not Available', 'value': 999},
                                                  
                                              ],
                                              #what in the box is wirtten before selecting any value
                                              placeholder="Select Job contract type",
                                              #style of the contract box
                                              style={
                                                  'width': '50%',
                                                  'height': '30px',
                                                  'lineHeight': '30px',
                                                  'lineWidth': '30px',
                                                  'display':'block',
                                                  "margin-left": "none",
                                                  "margin-right": "auto",
                                                  'borderWidth': '1px',
                                                  'borderStyle': 'dashed',
                                                  'borderRadius': '5px',
                                                  'textAlign': 'center'
                                                  },
                                              ),
                                 #creating the box of LFM segment
                                 html.H4(children='Enter value for LFM segment (calculated score between 110 and 429)'),
                                 dcc.Input(
                                     id="rfm", type="number",
                                     debounce=True, placeholder="Enter LFM",
                                 ),
                                 #creating the box of the vlaue of vehicles purchase based on province and date
                                 html.H4(children='Enter value Vehicles Purchases/Province (https://www.europeandataportal.eu/data/datasets/urn-ine3aes3atabla3at33a83a22356?locale=en)'),
                                 dcc.Input(
                                     id="var71",
                                     type="number",
                                     debounce=True, placeholder="Enter value",
                                 ),
                                 #creating the box of te value of ELectricity inflation based on province and date
                                 html.H4(children='Enter value for Electricity/Fuel/Gas Inflation percent (https://www.europeandataportal.eu/data/datasets/urn-ine3aes3atabla3at33a83a22356?locale=en)'
                                        ),
                                 dcc.Input(
                                     id="var45", type="number",
                                     debounce=True, placeholder="Enter value",
                                 ),
                                 
                                 
                             ]
                            ),
                    ]
                
               
               ),
        
              
       #tab where you are able to upload a new data set and calcualting the probility 
        dcc.Tab(
            label = 'Batch Prediction',
            children = [
                #title
                html.H1(children='Predict on uploaded test data'),
                html.Div(id = 'Upload_test_data_div',
                        children = [
                            dcc.Upload(
                                id="upload-data",
                                children=html.Div(
                                    #texst you see int eh box where you can upload a talbe
                                    ["Drag and drop or click to select a file to upload."]
                                    ),
                                #style of the tble
                                style={
                                    "width": "100%",
                                    "height": "60px",
                                    "lineHeight": "60px",
                                    "borderWidth": "1px",
                                    "borderStyle": "dashed",
                                    "borderRadius": "5px",
                                    "textAlign": "center",
                                    "margin": "10px",
                                    },
                                #make sure you are able to upload an oter data talbe
                                multiple=True,
                                ),
                        ]
                        ),
                html.Div(id='output-data-upload'),
                ]
        ),
    ]
    )
]
)







@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename'),
               State('upload-data', 'last_modified')])
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        
        return children


 
    
@app.callback(
    Output(component_id='sense_table_dropdown', component_property='data'),
    [Input(component_id='profit_margin', component_property='value')]
)
def update_data(col_id):
    data = sense_df[['AcceptanceRate', 'Threshold', col_id]].sort_values(col_id,ascending = False).to_dict('records')
    return data

    
@app.callback(
    Output(component_id='sense_table_dropdown',component_property= 'columns'),
    [Input(component_id='profit_margin', component_property='value')]
)
def update_columns(col_id):
    columns =[{"name": i, "id": i} for i in sense_df[['AcceptanceRate', 'Threshold', col_id]]]
    return columns

#creating a callback so where are able to output the pie cahrt with a correct value
@app.callback(
    Output(component_id='pred_pie', component_property='figure'),
    #all the value listed for making the prediction
    [Input('vivienda', 'value'),
     Input('COD_BQE_PQI_4_num', 'value'),
     Input('Origen', 'value'),
     Input('num_tit', 'value'),
     Input('edad', 'value'),
     Input('pais_cat', 'value'),
     Input('occu_cat', 'value'),
     Input('prov_cat', 'value'),
     Input('gender', 'value'),
     Input('email_domain', 'value'),
     Input('BLZ_OCCUPATION_1', 'value'),
     Input('ord_tipcontr1', 'value'),
     Input('rfm', 'value'),
     Input('var71', 'value'),
     Input('var45', 'value')
    ])
#the function that cralculate the probility
def datafrma(v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15):
    global df3, df4, df5, df6, df7, names
    #creating some empty dataframes
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    df5 = pd.DataFrame()
    df6 = pd.DataFrame()
    df7 = pd.DataFrame()
    #collection all the values that where input in one dataframe
    df3['vivienda'] = [v1]
    df3["COD_BQE_PAI_4"] = [v2]
    df3["Origen"] = [v3]
    df3["edad_1"] = [v4]
    df3["num_tit"] = [v5]    
    df3["pais_nac_1"] = [v6]
    df3["ocupacion_1"] = [v7]
    df3["Provincia"] = [v8]
    df3["sexo_1"] = [v9]
    df3["dominio_email_1"] = [v10]
    df3["BLZ_OCCUPATION_1"] = [v11]
    df3["ord_tipcontr1"] = [v12]
    df3["RFM_Segment"] = [v13]
    df3["71"] = [v14]
    df3["45"] = [v15]
    #only select the categorical varaibles and save in a sep dataframe
    df4 = df3[['vivienda','COD_BQE_PAI_4','Origen','pais_nac_1','ocupacion_1',
               'Provincia','sexo_1','dominio_email_1','BLZ_OCCUPATION_1','ord_tipcontr1',]]
    #all the other variables are stored in an other dataframe
    df5 = df3[['edad_1', 'num_tit','RFM_Segment', '71', '45']]
    #apply the woe function on the categorical variables
    df4 = woe_map2.transform(df4)
    #concat both tables 
    df6 = pd.concat([df4, df5],axis = 1)
    #concat and other table where other obersavtions where stored this way we are able to standerize the observation
    df7 = pd.concat([df6, test6])
    #standerize the data
    names = df7.columns
    df7 = pd.DataFrame(scale(df7),columns = names)
    #only select the observaion we need
    df7 = df7.iloc[:1,:]
    #calculate the probability for that observation
    df7['proba'] = pipeline.predict_proba(df7)[:,1] 
    #time.sleep(2)
    #creating labels and values for the pie chart
    labels = ['Probability of Default','Probability of Success']
    values = [df7['proba'].item() ,1-df7['proba'].item()]
    
    # Use `hole` to create a donut-like pie chart
    #displaying the pie chart
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.95)])
    return fig



#callback to display the sensivety table
@app.callback(
    Output(component_id='sense_div', component_property='children'),
    [Input(component_id='profit_margin', component_property='value')]
)
#function to make the chart
def update_data(col_id):
    
    fig = px.line(sense_df[['Threshold',col_id]], x="Threshold", y=col_id, title='Total Profit vs Threshold of Default Probability')
    children = [
                                 
                                 dcc.Graph(id = 'sense_line',
                                           figure = go.Figure(fig),
                                           #style of the graph
                                           style={
                                               'width': '85%',
                                               'height': '40%',
                                               'lineHeight': '60px',
                                               'lineWidth': '60px',
                                               'display':'block',
                                               'margin-left': 'auto',
                                               'margin-right': 'none',
                                               'borderWidth': '1px',
                                               'borderStyle': 'hidden',
                                               'borderRadius': '5px',
                                               'textAlign': 'center'
                                               },
                                           ),
                                 ]
    return children



@app.callback(
    Output(component_id='sense_div', component_property='style'),
    [Input(component_id='profit_margin', component_property='value')]
)
def update_data(col_id):
    style= {'width': '50%','display':'block', "margin-left": "auto","margin-right": "none",'right':'4%','top':'15%','vertical-align': 'text-top','position':'fixed'}
    
    return style




if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False,dev_tools_ui=False,dev_tools_props_check=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 116-848-108
Debugger PIN: 116-848-108
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
